## TensorFlow Version

In [1]:
import tensorflow as tf

In [5]:
x = tf.placeholder(tf.float32, [None, 224, 224, 3], name='input')


def inception_Block(inputs, branchShape0, branchShape1_1, branchShape1_2, branchShape2_1, branchShape2_2, branchShape3):
    """
    
    定义模块化结构
    """
    with tf.variable_scope('Branch_0'):
        branch_0 = tf.nn.conv2d(inputs, tf.Variable(tf.truncated_normal([1, 1, int(inputs.shape[3]), branchShape0])), strides=[1, 1, 1, 1], padding="SAME")
        branch_0 = tf.nn.bias_add(branch_0, tf.Variable(tf.constant(0.0, shape=[branchShape0])))
        branch_0 = tf.nn.relu(branch_0)
    with tf.variable_scope('Branch_1'):
        branch_1 = tf.nn.conv2d(inputs, tf.Variable(tf.truncated_normal([1, 1, int(inputs.shape[3]), branchShape1_1])), strides=[1, 1, 1, 1], padding="SAME")
        branch_1 = tf.nn.bias_add(branch_1, tf.Variable(tf.constant(0.0, shape=[branchShape1_1])))
        branch_1 = tf.nn.relu(branch_1)
        branch_1 = tf.nn.conv2d(branch_1, tf.Variable(tf.truncated_normal([3, 3, branchShape1_1, branchShape1_2])), strides=[1, 1, 1, 1], padding="SAME")
        branch_1 = tf.nn.bias_add(branch_1, tf.Variable(tf.constant(0.0, shape=[branchShape1_2])))
        branch_1 = tf.nn.relu(branch_1)
    with tf.variable_scope('Branch_2'):
        branch_2 = tf.nn.conv2d(inputs, tf.Variable(tf.truncated_normal([1, 1, int(inputs.shape[3]), branchShape2_1])), strides=[1, 1, 1, 1], padding="SAME")
        branch_2 = tf.nn.bias_add(branch_2, tf.Variable(tf.constant(0.0, shape=[branchShape2_1])))
        branch_2 = tf.nn.relu(branch_2)
        branch_2 = tf.nn.conv2d(branch_2, tf.Variable(tf.truncated_normal([5, 5, branchShape2_1, branchShape2_2])), strides=[1, 1, 1, 1], padding="SAME")
        branch_2 = tf.nn.bias_add(branch_2, tf.Variable(tf.constant(0.0, shape=[branchShape2_2])))
        branch_2 = tf.nn.relu(branch_2)
    with tf.variable_scope('Branch_3'):
        branch_3 = tf.nn.max_pool(inputs, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='SAME')
        branch_3 = tf.nn.conv2d(branch_3, tf.Variable(tf.truncated_normal([1, 1, int(inputs.shape[3]), branchShape3])), strides=[1, 1, 1, 1], padding="SAME")
        branch_3 = tf.nn.bias_add(branch_3, tf.Variable(tf.constant(0.0, shape=[branchShape3])))
        branch_3 = tf.nn.relu(branch_3)
    
    module_block = tf.concat(axis=3, values=[branch_0, branch_1, branch_2, branch_3])
    print('module_block :', module_block.shape)
    
    return module_block

def inception_V1(x):
    W_conv = {
    'conv1': tf.Variable(tf.truncated_normal([7, 7, 3, 64])),
    'conv2': tf.Variable(tf.truncated_normal([1, 1, 64, 192])),
    'conv3': tf.Variable(tf.truncated_normal([3, 3, 192, 192])),
    'fc1': tf.Variable(tf.truncated_normal([1024, 1000])),
    'fc2': tf.Variable(tf.truncated_normal([1000, 1000])),
    }

    b_conv = {
        'conv1': tf.Variable(tf.constant(0.0, shape=[64])),
        'conv2': tf.Variable(tf.constant(0.0, shape=[192])),
        'conv3': tf.Variable(tf.constant(0.0, shape=[192])),
        'fc1': tf.Variable(tf.constant(0.1, shape=[1000])),
        'fc2': tf.Variable(tf.constant(0.1, shape=[1000]))

    }

    conv1 = tf.nn.conv2d(x, W_conv['conv1'], strides=[1, 2, 2, 1], padding='SAME')
    conv1 = tf.nn.bias_add(conv1, b_conv['conv1'])
    conv1 = tf.nn.relu(conv1)
    print('conv1 :', conv1.shape)
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    print('pool1 :', pool1.shape)
    norm1 = tf.nn.lrn(pool1)
    
    conv2 = tf.nn.conv2d(norm1, W_conv['conv2'], strides=[1, 1, 1, 1], padding='VALID')
    conv2 = tf.nn.bias_add(conv2, b_conv['conv2'])
    conv2 = tf.nn.relu(conv2)
    print('conv2 :', conv2.shape)
    
    conv3 = tf.nn.conv2d(conv2, W_conv['conv3'], strides=[1, 1, 1, 1], padding='SAME')
    conv3 = tf.nn.bias_add(conv3, b_conv['conv3'])
    print('conv3 :', conv3.shape)
    norm2 = tf.nn.lrn(conv3)
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    print('pool2 :', pool2.shape)
    
    print('--------')   
    print('inception_3a')
    inception_3a = inception_Block(pool2, 64, 96, 128, 16, 32, 32)
    print('--------')
    print('inception_3b')
    inception_3b = inception_Block(inception_3a, 128, 128, 192, 32, 96, 64)    
    pool3 = tf.nn.max_pool(inception_3b, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    print('pool3 :', pool3.shape)
    
    print('--------')
    print('inception_4a')
    inception_4a = inception_Block(pool3, 192, 96, 208, 16, 48, 64)
    print('--------')
    print('inception_4b')
    inception_4b = inception_Block(inception_4a, 160, 112, 224, 24, 64, 64)
    print('--------')
    print('inception_4c')
    inception_4c = inception_Block(inception_4b, 128, 128, 256, 24, 64, 64)
    print('--------')
    print('inception_4d')
    inception_4d = inception_Block(inception_4c, 112, 144, 288, 32, 64, 64)
    print('--------')
    print('inception_4e')
    inception_4e = inception_Block(inception_4d, 256, 160, 320, 32, 128, 128)
    pool4 = tf.nn.max_pool(inception_4e, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME')
    print('pool4 :', pool4.shape)
    
    print('--------')
    print('inception_5a')
    inception_5a = inception_Block(pool4, 256, 160, 320, 32, 128, 128)
    print('--------')
    print('inception_5b')
    inception_5b = inception_Block(inception_5a, 384, 192, 384, 48, 128, 128)
    pool5 = tf.nn.avg_pool(inception_5b, ksize=[1, 7, 7, 1], strides=[1, 1, 1, 1], padding='VALID')
    print('pool5 :', pool5.shape)
    pool5 = tf.nn.dropout(pool5, 0.4)
    
    
    print('--------')
    reshape = tf.reshape(pool5, [-1, 1 * 1 * 1024])
    fc1 = tf.add(tf.matmul(reshape, W_conv['fc1']), b_conv['fc1'])
    fc1 = tf.nn.relu(fc1)
    print('fc1 :', fc1.shape)

    output = tf.nn.softmax(tf.add(tf.matmul(fc1, W_conv['fc2']), b_conv['fc2']))
    print('output :', output.shape)
    return 


In [6]:
inception_V1 = inception_V1(x)

conv1 : (?, 112, 112, 64)
pool1 : (?, 56, 56, 64)
conv2 : (?, 56, 56, 192)
conv3 : (?, 56, 56, 192)
pool2 : (?, 28, 28, 192)
--------
inception_3a
module_block : (?, 28, 28, 256)
--------
inception_3b
module_block : (?, 28, 28, 480)
pool3 : (?, 14, 14, 480)
--------
inception_4a
module_block : (?, 14, 14, 512)
--------
inception_4b
module_block : (?, 14, 14, 512)
--------
inception_4c
module_block : (?, 14, 14, 512)
--------
inception_4d
module_block : (?, 14, 14, 528)
--------
inception_4e
module_block : (?, 14, 14, 832)
pool4 : (?, 7, 7, 832)
--------
inception_5a
module_block : (?, 7, 7, 832)
--------
inception_5b
module_block : (?, 7, 7, 1024)
pool5 : (?, 1, 1, 1024)
--------
fc1 : (?, 1000)
output : (?, 1000)
